## Plot loop of clouds from .zarr file

In [14]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import geogif

In [18]:
zarr_path = '/storage/cdalden/goes/goes16/2023/'
zarr_file = '1june.zarr'

ds = xr.open_zarr(zarr_path + zarr_file)

# Define the bounds
lat_min, lat_max = 37, 41
lon_min, lon_max = -109, -104

# Create latitude and longitude arrays based on the bounds and the dimensions of the dataset
lat = np.linspace(lat_min, lat_max, ds.dims['y'])
lon = np.linspace(lon_min, lon_max, ds.dims['x'])

# Assign the latitude and longitude coordinates to the dataset
ds = ds.assign_coords(lon=('x', lon), lat=('y', lat))

# Verify the dataset
print(ds)

<xarray.Dataset> Size: 133MB
Dimensions:                                           (t: 288,
                                                       num_star_looks: 24,
                                                       y: 160, x: 241,
                                                       number_of_harmonization_coefficients: 3,
                                                       number_of_image_bounds: 2,
                                                       band: 1,
                                                       number_of_time_bounds: 2)
Coordinates:
    band_wavelength_star_look                         (num_star_looks) float32 96B dask.array<chunksize=(24,), meta=np.ndarray>
    y_image                                           float32 4B ...
  * y                                                 (y) float64 1kB 0.1092 ...
    t_star_look                                       (num_star_looks) datetime64[ns] 192B dask.array<chunksize=(24,), meta=np.ndarray>
    x_image 

/tmp/ipykernel_12608/3743806464.py:11: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  lat = np.linspace(lat_min, lat_max, ds.dims['y'])
/tmp/ipykernel_12608/3743806464.py:12: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  lon = np.linspace(lon_min, lon_max, ds.dims['x'])


In [22]:
variable = 'Rad'
# select our variable of interest
da = ds[variable]

# Create the gif animation with coordinates
gif_bytes = geogif.dgif(
    da,
    fps=20,
    cmap="Greys_r",
    date_format="%Y-%m-%d %H:%M:%S",
    date_position="ul",
    bytes=True,
).compute()

# write gif to file
with open(f"test1.gif", "wb") as f:
    f.write(gif_bytes)


In [21]:
geogif.dgif?

Signature:
geogif.dgif(
    arr: 'xr.DataArray',
    *,
    bytes=False,
    fps: 'int' = 16,
    robust: 'bool' = True,
    vmin: 'float | None' = None,
    vmax: 'float | None' = None,
    cmap: 'str | matplotlib.colors.Colormap | None' = None,
    date_format: 'str | None' = '%Y-%m-%d',
    date_position: "Literal['ul', 'ur', 'll', 'lr']" = 'ul',
    date_color: 'tuple[int, int, int]' = (255, 255, 255),
    date_bg: 'tuple[int, int, int] | None' = (0, 0, 0),
    date_size: 'int | float' = 0.15,
) -> 'Delayed'
Docstring:
Turn a dask-backed `~xarray.DataArray` timestack into a GIF, as a `~dask.delayed.Delayed` object.

The `~xarray.DataArray` must have 1 or 3 bands, and dimensions in
(``time``, [optional ``band``], ``y``, ``x``) order.

If all you want is a GIF, `dgif` can be faster than calling ``.compute()`` and then `gif`:
since GIFs are smaller and reduced in quality from NumPy arrays, there's less data to transfer
from the cluster back to your computer. Or, if you want to generat